In [1]:
# importing necessary packages
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

import json
import gc

from elasticsearch import Elasticsearch


In [2]:
# creating elasticsearch client
elastic_client = Elasticsearch('http://192.168.100.147:9200', request_timeout = 30, max_retries = 10, retry_on_timeout = True)


In [3]:
# constructing DSL query

min_timestamp = '2088-05-13T00:00:00' # '2088-05-13T00:00:00'
max_timestamp = '2088-05-13T10:00:00' # '2088-05-20T00:00:00'

json_body = {
              'query': {
                'bool': {
                  'must': [
                    {
                      'query_string': {
                        'query':'(host:4) and (host:12) and (host:15) and (host:19)'
                      }
                    },
                    {
                      'range': {
                            '@timestamp': {
                             'gte': min_timestamp, 
                             'lte': max_timestamp, 
                             'boost': 2.0
                         }
                      }
                    }
                  ]
                }
              }
            }

# json_body = {
#              'query': {
#                  'range': {
#                      '@timestamp': {
#                          'gte': min_timestamp, 
#                          'lte': max_timestamp, 
#                          'boost': 2.0
#                      }
#                  }
#              }
#          }

# Lucene type query
# lucene_q='(host:4) and (host:12) and (host:15) and (host:19)'
# choose live-TV host 2, 7; VoD host 12, 4

In [4]:
# trying to get count of records returned from Lucene query format
rec_count2 = elastic_client.count(body=json_body, index='batch3-*') # q=lucene_q

# observing the actual output from client.count
rec_count2


/tmp/ipykernel_4014/4106050485.py:2: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  rec_count2 = elastic_client.count(body=json_body, index='batch3-*') # q=lucene_q
/tmp/ipykernel_4014/4106050485.py:2: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  rec_count2 = elastic_client.count(body=json_body, index='batch3-*') # q=lucene_q


ObjectApiResponse({'count': 23315, '_shards': {'total': 640, 'successful': 640, 'skipped': 0, 'failed': 0}})

In [5]:
# trying to get count of records returned from DSL query format
rec_count = elastic_client.count(body=json_body, index='batch3-*')

# extracting only the 'count' attribute from the output
rec_count['count']


/tmp/ipykernel_4014/77014873.py:2: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  rec_count = elastic_client.count(body=json_body, index='batch3-*')
/tmp/ipykernel_4014/77014873.py:2: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  rec_count = elastic_client.count(body=json_body, index='batch3-*')


23315

### parallel processing to get the dataset

In [11]:
# function for multiprocessing scrolling API to get dataset from elasticsearch {modified piece}
'''
this code works wonderfully!
'''

# all_cols = ['devicefamily','livechannel','fragment','path','popname','protocol','timefirstbyte','@timestamp','coordinates',
#             'contentlength','assetnumber','timetoserv','contenttype','maxage','@version','sid','cachecontrol','method',
#             'hit','devicemodel','cachename','uid','host','contentpackage','devicebrand','osfamily','statuscode','uafamily',
#             'uamajor','manifest']

# attributes = ['@timestamp','statuscode','contentlength','host','timetoserv','hit','contenttype',
#               'uid','sid','livechannel','contentpackage','assetnumber','coordinates',
#               'uafamily']

def es_scroll(index):
    es = Elasticsearch('http://192.168.100.147:9200', request_timeout = 30, max_retries = 10, retry_on_timeout = True)
    page = es.search(index = index,
                     # doc_type = 'tls_book',
                     scroll = '2m',
                     _source=attributes,
                     body = json_body,
                     # q = lucene_q,
                     #version = False,
                     size = 10000
                    )
    sid = page['_scroll_id']
    # print('sid:', sid)
    scroll_size = len(page['hits']['hits'])
    
    print('records:', page['hits']['hits'])
    
    # Start scrolling
    df = pd.DataFrame()
    appended_data = []
    
    print('records returned from client.search:', scroll_size)
    
    while (scroll_size > 0):
        frame = pd.DataFrame.from_dict([document['_source'] for document in page['hits']['hits']])
        appended_data.append(frame)
        page = es.scroll(scroll_id = sid, scroll = '2m', request_timeout = 30)
        # Update the scroll ID
        sid = page['_scroll_id']
        # Get the number of results that we returned in the last scroll
        scroll_size = len(page['hits']['hits'])
        
    print('size of appended data:', len(appended_data))
    if len(appended_data) > 0:
        df = pd.concat(appended_data, ignore_index = True, sort = False)
    del appended_data
    gc.collect()
    es.clear_scroll(body = {'scroll_id': sid})
    return df

In [12]:
vod_df = es_scroll(index='batch3-*')

/tmp/ipykernel_4014/1852347243.py:17: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  page = es.search(index = index,
/tmp/ipykernel_4014/1852347243.py:17: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  page = es.search(index = index,
/tmp/ipykernel_4014/1852347243.py:41: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  page = es.scroll(scroll_id = sid, scroll = '2m', request_timeout = 30)
IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerAp

size of appended data: 3


/tmp/ipykernel_4014/1852347243.py:52: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.clear_scroll(body = {'scroll_id': sid})
/tmp/ipykernel_4014/1852347243.py:52: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.clear_scroll(body = {'scroll_id': sid})


In [ ]:
print(vod_df.shape)
vod_df

In [ ]:
vod_df.info()

In [ ]:
vod_df.to_csv('CSV/VoD_ELK_24_3.csv', index = False)

In [ ]:
# function for multiprocessing scrolling API to get dataset from elasticsearch {original piece}
# 
# def es_scroll(index, min_timestamp, max_timestamp, slice_no):
#     es = Elasticsearch('http://192.168.100.146:9200', timeout = 30, max_retries = 10, retry_on_timeout = True)
#     page = es.search(index = index,
#                      doc_type = 'tls_book',
#                      scroll = '1m',
#                      body = {
#                          'slice': {
#                              'id': slice_no,
#                              'max': SLICES
#                          },
#                          '_source': [
#                              'host', 'contentpackage'
#                          ],
#                          'sort': [
#                              '_doc'
#                          ],
#                          'query': {
#                              'range': {
#                                  '@timestamp': {
#                                      'gte': min_timestamp, or 'gt': '2088-05-13T00:00:00'
#                                      'lte': max_timestamp, or 'lt': '2088-05-20T00:00:00'
#                                      'boost': 2.0
#                                  }
#                              }
#                          }
#                      },
#                      version = False,
#                      size = 10000
#                     )
#     sid = page['_scroll_id']
#     scroll_size = page['hits']['hits']
    
#     # Start scrolling
#     df = pd.DataFrame()
#     appended_data = []
    
#     while (scroll_size > 0):
#         frame = pd.DataFrame.from_dict([document['_source'] for document in page ['hits']['hits']])
#         appended_data.append(frame)
#         page = es.scroll(scroll_id = sid, scroll = '1m', request_timeout = 30)
#         # Update the scroll ID
#         sid = page['_scroll_id']
#         # Get the number of results that we returned in the last scroll
#         scroll_size = len(page['hits']['hits'])
#     if len(append_data) > 0:
#         df = pd.concat(appended_data, ignore_index = True, sort = False)
#     del appended_data
#     gc.collect()
#     es.clear_scroll(body = {'scroll_id': sid})
#     return df
    
    
    

In [ ]:
# create a JSON string from Pandas object
# json_data = elastic_df.to_json()
# print('\nto_json() method:', json_data)

In [ ]:
# verify that the to_jason() method made a JSON string
# try:
#     json.loads(json_data)
#     print('\njson_data is a valid JSON string')
# except json.decoder.JSONDecodeError as err:
#     print('\njson.decoder.JSONDecodeError:', err)
#     print('json_data is NOT a valid JSON string')



In [ ]:
'''
Creating Series object from Elasticsearch documents
'''

# # an empty dictionary for series arrays
# elastic_series = {}
# #initializing new indexer for the list of elastic_docs
# j = 0

# # iterate the docs returned from API call
# while(j < len(elastic_docs)):
#     for num, doc in enumerate(elastic_docs[j]):

#         # get the _id for the doc
#         _id = doc['_id']

#         # get the source data from document
#         source_data = doc['_source']

#         # make a Series object for the doc using _id as key
#         elastic_series[_id] = Series()

#         # iterate source_data
#         for field, value in source_data.items():

#             # set the field type as Series index and value as Series value
#             elastic_series[_id].at[field] = value
#     j += 1
        
# for key, doc in elastic_series.items():
#     print('\nID:', key, '\n', doc)